# Todos

## General

- ✅ bale ta ballast trips apo ships tensor pisw sto ships df
- ✅ tsekare me mock random data oti ta functions gia cii calculation apo speed trexoun komple

## Step
    
**Pws ypologizw to `reward` apo to action**
    
- ✅ Value of contract
- ✅ CII
- ✅ Lateness
- ✅ Ftiakse sugentrwtiko `calculate_reward(selected_ship_idx,selected_contract,selected_speed)` function
        
**Pws na kanw swsta update to `state`**
        
- ❌ contract tensor
- ❌ ship tensor
- ❌ contract mask
- ❌ ship mask
- ❌ ship_log
        

## Train 

ti einai ta `advantages`, `returns` kai giati xrhsimopoioyntai:
        
- ❌ sthn baseline_net.update(observations, returns) gia to update to baselineNet
- ❌ sthn PolicyGradient.update_policy(observations, actions, advantages)gia to update ths policygradient 

# Code

In [1]:
# autoreloading modules that i change in vscode
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [3]:
# load data
data_dict = {
    "fleet_path": "data/fleet_small.csv",
    "ports_path": "data/ports_10.csv",
    "dm_path": "data/distance_matrix.csv",
}


ports_df = pd.read_csv(data_dict["ports_path"])
fleet_df = pd.read_csv(data_dict["fleet_path"])
distance_matrix = pd.read_csv(data_dict["dm_path"])

In [4]:
from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from utils.utils import map_action

In [ ]:
# run this cell when needing to reload the classes CarbonEnv, BaselineNet, PolicyNet

del sys.modules['models.models']
del sys.modules['env.env']

from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from utils.utils import map_action

In [27]:
envo = CarbonEnv()

In [28]:
envo.contracts_df

,start_port_number,end_port_number,contract_type,start_day,end_day,cargo_size,contract_duration,contract_availability,contract_distance,value
0,5.0,10.0,4.0,0.0,4.0,71176.0,5.0,1.0,425.0,6.0
1,10.0,8.0,4.0,0.0,4.0,72997.0,5.0,1.0,342.0,5.0
2,7.0,9.0,4.0,0.0,5.0,74859.0,6.0,1.0,688.0,9.0
3,4.0,10.0,1.0,0.0,7.0,41456.0,8.0,1.0,1029.0,5.0


In [29]:
envo.ships

,ship_number,dwt,cii_threshold,cii_attained,current_port,current_speed,ship_availability,ballast_1,ballast_2,ballast_3,ballast_4
0,1,50000,5.385183,0,8,0,1,83.0,342.0,1140.0,1256.0
1,2,60000,4.807837,0,6,0,1,1183.0,876.0,502.0,356.0
2,3,70000,4.368264,0,2,0,1,1543.0,1621.0,2503.0,2619.0
3,4,80000,4.020109,0,9,0,1,1480.0,1174.0,688.0,198.0


In [ ]:
# something
s = np.array([[0], [0], [0], [1]])


k = np.where(s==0,3*[False],3*[True])

k

k.ravel()

st = tf.convert_to_tensor(s)
st

bm = tf.equal(st,0)


skou = tf.where(bm,tf.repeat(tf.constant(False),3),tf.repeat(tf.constant(True),3))
skou

skou = tf.reshape(skou,[-1])
skou

skou = tf.expand_dims(skou,axis=1)
skou

bol = tf.constant(False,shape=(1,1))

tf.concat((skou,bol),axis=0)

In [9]:
initial_state = envo.state

In [8]:
envo.ship_log

{1: 0, 2: 0, 3: 0, 4: 0}

In [ ]:
initial_state['ships_state'][:,-4:].numpy()

In [10]:
baseline_net = BaselineNet(embedding_size=envo.embedding_size)

In [11]:
policy_net = PolicyNet(embedding_size=envo.embedding_size,output_size=envo.action_space_dim[0])

In [12]:
logits, distr = policy_net.action_distribution(initial_state)

In [13]:
action = policy_net.sample_action(initial_state)

In [14]:
action

3

In [15]:
# estw oti dialegw to ship 1 mesa sth loopa twn ships
# h loopa twn ships einai prin to sample_action

ship_number = 1
ship_idx = ship_number - 1

In [31]:
envo.ships

,ship_number,dwt,cii_threshold,cii_attained,current_port,current_speed,ship_availability,ballast_1,ballast_2,ballast_3,ballast_4
0,1,50000,5.385183,0,8,0,1,83.0,342.0,1140.0,1256.0
1,2,60000,4.807837,0,6,0,1,1183.0,876.0,502.0,356.0
2,3,70000,4.368264,0,2,0,1,1543.0,1621.0,2503.0,2619.0
3,4,80000,4.020109,0,9,0,1,1480.0,1174.0,688.0,198.0


In [32]:
envo.contracts_df

,start_port_number,end_port_number,contract_type,start_day,end_day,cargo_size,contract_duration,contract_availability,contract_distance,value
0,5.0,10.0,4.0,0.0,4.0,71176.0,5.0,1.0,425.0,6.0
1,10.0,8.0,4.0,0.0,4.0,72997.0,5.0,1.0,342.0,5.0
2,7.0,9.0,4.0,0.0,5.0,74859.0,6.0,1.0,688.0,9.0
3,4.0,10.0,1.0,0.0,7.0,41456.0,8.0,1.0,1029.0,5.0


In [51]:
x = envo.step(action=0,ship_number=1)

The contract selected is contract_0 and the speed selected for ship 1 is 10 knots
Bhka sto if, phra contract ! :D
To reward sthn arxh einai 0
To start port tou contract_0 einai to 5.0
To end port tou contract_0 einai to 10.0
H apostash metaksy twn dyo autwn port einai 425.0 nm
To ballast_idx pou epileksame einai to ballast_0 pou einai to feature 7 tou ship_tensor
To ploio 1 brisketai sto current port 10.0
To ballast distance metaksy current port 10.0 kai start port 5.0 prokyptei 83.0 nm
To synoliko distance einai 425.0 nm + 83.0 nm = 508.0 nm
To ship 1 8a ftasei 3.0 meres nwritera
to distance sailed einai 508.0
To dwt einai 50000
O c3 speed factor einai 0.00085034
O c2 speed factor einai 0.15646258
Ta co2 emissions einai 16.496598
O ari8mhths einai 16496597.999999998
O paronomasths einai 25400000.0
The total reward is made up by the contract value 6.0
The delay reward is 3.0
The cii reward is 3.7010552883148193
tf.Tensor(1.2989447, shape=(), dtype=float32)
ekana update to state
tsekare

In [45]:
int(16496597.999999998/ 25400000.0)

0

In [49]:
envo.contracts_tensor[0,:]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([5.0000e+00, 1.0000e+01, 4.0000e+00, 0.0000e+00, 4.0000e+00,
       7.1176e+04, 5.0000e+00, 0.0000e+00, 4.2500e+02, 6.0000e+00],
      dtype=float32)>

In [40]:
envo.contracts_df.iloc[0,1]

10.0

In [50]:
envo.ships_tensor[0,3:7]

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 0.64947236, 10.        , 10.        ,  0.        ], dtype=float32)>

In [18]:
contract, speed = map_action(selected_action=0)

In [19]:
contract, speed

(0, 10)

In [24]:
envo.contracts_tensor[0,:].numpy()

array([5.0000e+00, 1.0000e+01, 4.0000e+00, 0.0000e+00, 4.0000e+00,
       7.1176e+04, 5.0000e+00, 1.0000e+00, 4.2500e+02, 6.0000e+00],
      dtype=float32)

In [53]:
state_dict = {
            "contracts_state": envo.contracts_tensor,
            "ships_state": envo.ships_tensor,
            "contracts_mask": envo.contracts_mask,
            "ships_mask": envo.ships_mask,
        }


In [76]:
list(state_dict.values())

[<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
 array([[5.0000e+00, 1.0000e+01, 4.0000e+00, 0.0000e+00, 4.0000e+00,
         7.1176e+04, 5.0000e+00, 0.0000e+00, 4.2500e+02, 6.0000e+00],
        [1.0000e+01, 8.0000e+00, 4.0000e+00, 0.0000e+00, 4.0000e+00,
         7.2997e+04, 5.0000e+00, 1.0000e+00, 3.4200e+02, 5.0000e+00],
        [7.0000e+00, 9.0000e+00, 4.0000e+00, 0.0000e+00, 5.0000e+00,
         7.4859e+04, 6.0000e+00, 1.0000e+00, 6.8800e+02, 9.0000e+00],
        [4.0000e+00, 1.0000e+01, 1.0000e+00, 0.0000e+00, 7.0000e+00,
         4.1456e+04, 8.0000e+00, 1.0000e+00, 1.0290e+03, 5.0000e+00]],
       dtype=float32)>,
 <tf.Tensor: shape=(4, 11), dtype=float32, numpy=
 array([[1.0000000e+00, 5.0000000e+04, 5.0000000e+00, 1.2989447e+00,
         1.0000000e+01, 1.0000000e+01, 0.0000000e+00, 8.3000000e+01,
         3.4200000e+02, 1.1400000e+03, 1.2560000e+03],
        [2.0000000e+00, 6.0000000e+04, 4.8078370e+00, 0.0000000e+00,
         6.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.1

In [63]:
state_dict

{'contracts_state': <tf.Tensor: shape=(4, 10), dtype=float32, numpy=
 array([[5.0000e+00, 1.0000e+01, 4.0000e+00, 0.0000e+00, 4.0000e+00,
         7.1176e+04, 5.0000e+00, 0.0000e+00, 4.2500e+02, 6.0000e+00],
        [1.0000e+01, 8.0000e+00, 4.0000e+00, 0.0000e+00, 4.0000e+00,
         7.2997e+04, 5.0000e+00, 1.0000e+00, 3.4200e+02, 5.0000e+00],
        [7.0000e+00, 9.0000e+00, 4.0000e+00, 0.0000e+00, 5.0000e+00,
         7.4859e+04, 6.0000e+00, 1.0000e+00, 6.8800e+02, 9.0000e+00],
        [4.0000e+00, 1.0000e+01, 1.0000e+00, 0.0000e+00, 7.0000e+00,
         4.1456e+04, 8.0000e+00, 1.0000e+00, 1.0290e+03, 5.0000e+00]],
       dtype=float32)>,
 'ships_state': <tf.Tensor: shape=(4, 11), dtype=float32, numpy=
 array([[1.0000000e+00, 5.0000000e+04, 5.0000000e+00, 1.2989447e+00,
         1.0000000e+01, 1.0000000e+01, 0.0000000e+00, 8.3000000e+01,
         3.4200000e+02, 1.1400000e+03, 1.2560000e+03],
        [2.0000000e+00, 6.0000000e+04, 4.8078370e+00, 0.0000000e+00,
         6.0000000e+00,

In [ ]:
indices_array = np.zeros(11)

In [ ]:
indices_array[[3,4,5,6]] = 1

In [ ]:
indices_array

In [ ]:
selected_contract_tensor[1].numpy()

In [ ]:
selected_ship_tensor = envo.ships_tensor[ship_idx,:]
selected_ship_tensor[]

In [ ]:
updates = tf.reshape(selected_ship_tensor, shape = (1, selected_ship_tensor.shape[0]))
updates

In [ ]:
tensor = envo.ships_tensor
tensor

In [ ]:
indices = tf.constant([[0,1],
                       [2,2],
                       [3,4]])
indices

In [ ]:
num_updates, index_depth = indices.shape.as_list()
num_updates, index_depth

In [ ]:
tensor.shape

In [ ]:
outer_shape = tensor.shape[:index_depth]
inner_shape = tensor.shape[index_depth:]
outer_shape, inner_shape

In [ ]:
tensor.shape

# outer_shape = 4
# inner_shape = 11

In [ ]:
updates.shape

In [ ]:
num_updates

In [ ]:
inner_shape

In [ ]:
action

In [ ]:
envo.step(action,ship=1)

In [ ]:
envo.step(0,2)

In [ ]:
envo.ships_tensor[:,0]

In [ ]:
envo.ships_tensor

In [ ]:
tensor = tf.zeros([6,3], dtype=tf.int32)
indices = tf.constant([[2],[4]])
num_updates, index_depth = indices.shape.as_list()
print(f"num_updates == {num_updates}, index_depth == {index_depth}")
print("")

print("The outer_shape should be 6 and the inner_shape should be 3")
print("")
      
print("If we use the definition of the first code block")
outer_shape_1st = tensor.shape[index_depth:]
inner_shape_1st = tensor.shape[:index_depth]
print(f"outer_shape is {outer_shape_1st[0]}, the inner_shape is {inner_shape_1st[0]}")      
print("")


print("If we use the definition of the forth code block")
outer_shape_4th = tensor.shape[:index_depth]
inner_shape_4th = tensor.shape[index_depth:]
print(f"outer_shape is {outer_shape_4th[0]}, the inner_shape is {inner_shape_4th[0]}")

### Tensorflow Tip

Assign values to a tensor at specific indices

In [ ]:
input_array = np.array([2, 4, 7, 11, 3, 8, 9, 19, 11, 7])
inplace_array = np.array([10, 20])
indices_array = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

In [ ]:
# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

In [ ]:
# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

In [ ]:
# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

In [ ]:
# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type